In [1]:
from __future__ import print_function
from pan_allele_data_helpers import *
import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.metrics import roc_auc_score, accuracy_score
log_transformed_ic50_cutoff = 1 - np.log(500)/np.log(5000)
from keras.models import Graph
import theano.tensor as T
from keras.models import Sequential, Graph
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Activation, RepeatVector, Dropout, Reshape, Flatten, Merge, Permute
import numpy as np
import theano
from keras.layers.convolutional import Convolution1D, MaxPooling1D
import collections
import random
from five_fold_validation import split_train_test

Couldn't import dot_parser, loading of dot files will not be possible.


#Get 10% of training set to select epoch

In [28]:
from pan_allele_data_helpers import *

path = "/Users/NanditaD/Intern/mhclearn"
mhc_sequence_fasta_file=path+"/py/pan_allele/files/pseudo/pseudo_sequences.fasta"
iedb_data_file= path + "/py/pan_allele/files/bdata.2009.mhci.public.1.txt"
with open(iedb_data_file, "rb") as source:
    lines = [line for line in source]
print(len(lines))
lines = np.array(lines[1:])
arr = np.arange(len(lines))
np.random.shuffle(arr)
train, test = split_train_test(arr, 10, 2)
print (len(train), len(test), len(lines[train]))
print(lines[train])
with open(path + "/py/pan_allele/files/2009_test_10.txt", "wb") as sink_test:
    print ("species\tmhc\tpeptide_length\tcv\tsequence\tinequality\tmeas", file=sink_test)
    sink_test.write("".join(lines[test]))
with open(path + "/py/pan_allele/files/2009_train_10.txt", "wb") as sink_train:
    print ("species\tmhc\tpeptide_length\tcv\tsequence\tinequality\tmeas", file=sink_train)
    sink_train.write("".join(lines[train]))

137655
123888 13766 123888
['human\tHLA-B-0802\t9\tTBD\tYHLGGIEGL\t>\t20000.0\n'
 'human\tHLA-B-4501\t10\tTBD\tVPLDEDFRKY\t>\t70000.0\n'
 'human\tHLA-A-0201\t9\tTBD\tCMLDGGNML\t=\t1041.666667\n' ...,
 'human\tHLA-A-3002\t9\tTBD\tAYLRKHFSM\t>\t75000.0\n'
 'macaque\tMamu-A-11\t9\tTBD\tAENLKVTVY\t=\t1065.861894\n'
 'human\tHLA-A-1101\t9\tTBD\tGSDGGLDDY\t>\t20000.0\n']


In [29]:
allele_groups_test, df_test = load_binding_data("files/2009_test_10.txt")
allele_sequence_data, max_allele_length = load_allele_sequence_data(mhc_sequence_fasta_file)
alleles_test = create_allele_list(allele_groups_test, allele_sequence_data)
print (len(df_test))
allele_groups_train, df_train = load_binding_data('files/2009_train_10.txt')
alleles_train = create_allele_list(allele_groups_train, allele_sequence_data)
print (len(df_train))

9047
81916


#Get 2013 data - 2009

In [2]:
def normalize_allele_name(allele_name):
    allele_name = allele_name.upper()
    # old school HLA-C serotypes look like "Cw"
    allele_name = allele_name.replace("CW", "C")
    patterns = [
        "HLA-",
        "-",
        "*",
        ":"
    ]
    for pattern in patterns:
        allele_name = allele_name.replace(pattern, "")
    return allele_name
allele_groups_2013, df_2013 = load_binding_data('files/bdata.2013.mhci.public.1.txt')
allele_sequence_data, max_allele_length = load_allele_sequence_data('files/trimmed-human-class1.fasta')
allele_list_2013 = sorted(create_allele_list(allele_groups_2013, allele_sequence_data))
pep_dict  = collections.defaultdict(dict)
for allele in allele_list_2013:
    #print allele, np.sum(allele_groups_2013[allele][2] < 500),'\t', (allele_groups_2013[allele][2] < 500).size
    for idx, peptide in enumerate(allele_groups_2013[allele][1]):
        pep_dict[allele][peptide] = allele_groups_2013[allele][2][idx]
        
    #if (allele_groups[allele][2]).size > 1000:
        #print allele, np.sum(allele_groups[allele][2] < 500),'\t', (allele_groups[allele][2] < 500).size
    

In [3]:
allele_groups_2009, df_2009 = load_binding_data('files/bdata.2009.mhci.public.1.txt')
allele_list_2009 = sorted(create_allele_list(allele_groups_2009, allele_sequence_data))
#for allele in allele_list:
#    print allele, np.sum(allele_groups[allele][2] < 500),'\t', (allele_groups[allele][2] < 500).size
    #if (allele_groups[allele][2]).size > 1000:
        #print allele, np.sum(allele_groups[allele][2] < 500),'\t', (allele_groups[allele][2] < 500).size
    

In [5]:
f = open("files/bdata2013-2009.txt",'w')
print ("species\tmhc\tpeptide_length\tcv\tsequence\tinequality\tmeas", file=f)
for allele in allele_list_2013:
    #print allele, len(allele_groups_2009[allele][2]),'\t', len(allele_groups_2013[allele][2])
    allele_groups_2013, df_2013 = load_binding_data('files/bdata.2013.mhci.public.1.txt')
    peptides_2013 = allele_groups_2013[allele][1]
    try:
        allele_groups_2009, df_2009 = load_binding_data('files/bdata.2009.mhci.public.1.txt')
        peptides_2009 = allele_groups_2009[allele][1]
        intersection = list(set(peptides_2009).intersection(peptides_2013))
        [peptides_2013.remove(peptide) for peptide in intersection]
        for peptide in peptides_2013:
            print ("human\t"+allele+"\t9\tTBD\t"+peptide+"\t=\t"+str(pep_dict[allele][peptide]),file=f) 
        #print peptides_2009, peptides_2013,allele_groups_2009[allele][1]
        #print allele,'\t', len(intersection),'\t', len(peptides_2009),'\t', len(peptides_2013)
    except Exception as e: 
        for peptide in peptides_2013:
            print ("human\t"+allele+"\t9\tTBD\t"+peptide+"\t=\t"+str(pep_dict[allele][peptide]),file=f) 
    
#print allele_groups_2009['A0210'][1], allele_groups_2013['A0210'][1]    

#Allele occurences

In [6]:
allele_groups, df = load_binding_data('files/bdata.2009.mhci.public.1.txt')
for allele in sorted(allele_groups.keys()):
    print (allele, np.sum(allele_groups[allele][2] < 500),'\t', (allele_groups[allele][2] < 500).size)
 

A0101 391 	 3169
A0201 2198 	 6961
A0202 1073 	 2314
A0203 1278 	 3937
A0205 31 	 36
A0206 1267 	 3223
A0207 7 	 30
A0210 0 	 18
A0211 361 	 1038
A0212 275 	 1143
A0216 160 	 894
A0219 204 	 1203
A0250 88 	 132
A0301 958 	 4601
A0302 2 	 2
A11 31 	 31
A1101 1139 	 3862
A2 18 	 30
A2301 291 	 1513
A2402 322 	 1985
A2403 241 	 1165
A2501 66 	 519
A26 4 	 4
A2601 260 	 2395
A2602 67 	 202
A2603 25 	 205
A2902 470 	 1839
A3001 569 	 1949
A3002 235 	 912
A3101 681 	 3309
A3201 275 	 575
A3301 224 	 1616
A6601 4 	 4
A6801 641 	 1700
A6802 643 	 3188
A6901 222 	 2079
A8001 113 	 782
B0702 566 	 2974
B0801 458 	 2084
B0802 19 	 486
B0803 9 	 217
B1402 0 	 3
B1501 830 	 3142
B1502 124 	 164
B1503 332 	 416
B1509 16 	 346
B1517 271 	 846
B1801 186 	 1661
B2701 1 	 1
B2702 0 	 4
B2703 0 	 433
B2705 351 	 2324
B3501 505 	 1993
B3503 1 	 5
B3801 3 	 136
B3901 170 	 879
B4001 282 	 2326
B4002 164 	 474
B4201 2 	 2
B4402 105 	 1295
B4403 101 	 502
B4501 103 	 483
B4601 91 	 1406
B4801 68 	 861
B5101 

In [2]:

sequence_array =[]
allele_list = []
if(remove_residues):
    with open(path + '/py/pan_allele/files/trimmed-human-class1-IEDB.fasta','rU') as f:
        for record in SeqIO.parse(f, 'fasta'):
            name, sequence = record.description, str(record.seq)
            name = name.split(' ')[0]
            sequence_array.append(sequence)
            allele_list.append(name)



    sequence_mat = np.array([list(seq) for seq in sequence_array])
    delete_columns = []

    for columns in range(0,sequence_mat.shape[1]):
        char_occurence_dict =  Counter(sequence_mat[:,columns])
        print char_occurence_dict

NameError: name 'remove_residues' is not defined